In [1]:
import pandas as pd
import matplotlib.pyplot as plt

### Loan_Information.csv: data providing information related to loans.

 - UID - customer ID
 - Recordnumber - customer’s loan id
 - Openbalance - loan opening balance
 - Accstartdate - account opening date
 - First_month - month when customer first logged into online servicing
 - Last_month - month when customer last logged into online servicing
 - Repayperiod - original loan term
 - Searchdate - date when customer data was uploaded to the database

In [2]:
data = pd.read_csv ('../../../assignment_datasets/Loan_Information.csv')
data.columns = pd.Series (data.columns).astype (str).apply (lambda x : x.lower ()).values

In [3]:
data.shape

(58687, 8)

In [4]:
data.columns

Index(['uid', 'recordnumber', 'openbalance', 'accstartdate', 'first_month',
       'last_month', 'repayperiod', 'searchdate'],
      dtype='object')

In [5]:
# same number of IDs as other data sets

data.uid.drop_duplicates ().shape

(16347,)

In [6]:
# One entry per UID and record number

duplicate_entries = data [['uid', 'recordnumber']].value_counts ()
duplicate_entries [duplicate_entries > 1]

Series([], dtype: int64)

In [7]:
data.head ()

,uid,recordnumber,openbalance,accstartdate,first_month,last_month,repayperiod,searchdate
0,784c964e-d3e1-4dd3-af6c-4288c4b37d27,1,600.0,2014-09-29,2014-10-01,2016-07-01,24.0,20181011
1,fc30d99d-106a-44bf-bc33-9a95f0d2d258,6,500.0,2014-01-24,2014-02-01,2014-07-01,6.0,20180524
2,fc30d99d-106a-44bf-bc33-9a95f0d2d258,9,1149.0,2013-10-18,2013-11-01,2018-05-01,NaN,20180524
3,fc30d99d-106a-44bf-bc33-9a95f0d2d258,14,446.0,2015-07-01,2015-08-01,2015-12-01,5.0,20180524
4,89761744-b42f-43cd-bd3c-5c363c44671a,1,1492.0,2014-10-06,2014-11-01,2015-11-01,13.0,20190924


In [8]:
data.searchdate = (data.searchdate.astype (str).apply (lambda x : x [:4]) + '-' + 
                   data.searchdate.astype (str).apply (lambda x : x [4:6])  + '-' +
                   data.searchdate.astype (str).apply (lambda x : x [6:]))

In [9]:
data.accstartdate = pd.to_datetime (data.accstartdate, yearfirst=True)
data.first_month = pd.to_datetime (data.first_month, yearfirst=True)
data.last_month = pd.to_datetime (data.last_month, yearfirst=True)
data.searchdate = pd.to_datetime (data.searchdate, yearfirst=True)

In [10]:
# Potential feature? Number of loans per id?

number_of_loans = (data ['uid']
                   .value_counts ()
                   .reset_index ()
                   .rename (columns = {'index' : 'uid', 
                                       'uid' : 'total_loan_number'}))

number_of_loans [number_of_loans.total_loan_number > 1].shape

(13013, 2)

In [11]:
# Hmmm, there's some thinking to do here—some assumptions to be made about the loan type

data [data.repayperiod.isna ()].shape

(2720, 8)

In [12]:
null_repay_data = (data [data.repayperiod.isna ()]
                   .uid
                   .value_counts ()
                   .reset_index ()
                   .rename (columns = {'index' : 'uid', 
                                       'uid' : 'no_repay_loan_number'})
                  ).fillna (0)

In [14]:
data_valid = data [(data.repayperiod >= 12) & (data.repayperiod <= 60)]

number_of_valid_loans = (data_valid ['uid']
                         .value_counts ()
                         .reset_index ()
                         .rename (columns = {'index' : 'uid', 
                                             'uid' : 'total_valid_loan_number'}))

In [15]:
number_of_loans = (number_of_loans
                   .join (number_of_valid_loans.set_index ('uid'), on = 'uid')
                   .join (null_repay_data.set_index ('uid'), on = 'uid')
                  ).fillna (0)

In [16]:
number_of_loans.to_csv ('../../data/loans_per_customer.csv', index = False)

In [17]:
data.to_csv ('../../data/loan_information.csv', index = False)